In [1]:
#!pip install langchain langchain-community langchain-core langchain-google-genai sentence-transformers scikit-learn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os, time
import numpy as np

In [3]:
textos_teste = [
    "Qual é a política de férias da nossa empresa?",
    "preciso de um relatório de despesas de viagem.",
    "Como configuro o acesso à rede privada virtual (VPN)?",
    "Onde encontro o código de conduta da organização?",
    "Quero entender o processo de avaliação de performance."
]

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

#### 1. Gemini

In [7]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

start_time = time.time()
embeddings_gemini = gemini_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos.")
print(f"- Dimensão do vetor: {len(embeddings_gemini[0])}.")

Tempo de processamento: 1.5112802982330322 segundos.
- Dimensão do vetor: 3072.


#### 2. Hugging Face - MiniLM

In [9]:
minilm_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

start_time = time.time()
embeddings_minilm = minilm_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos.")
print(f"- Dimensão do vetor: {len(embeddings_minilm[0])}.")

C:\Users\ana\Documents\Cursos\aluraDataScience\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tempo de processamento: 0.4548335075378418 segundos.
- Dimensão do vetor: 384.


#### 3. Hugging Face - BGE

In [10]:
bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

start_time = time.time()
embeddings_bge = bge_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time} segundos.")
print(f"- Dimensão do vetor: {len(embeddings_bge[0])}.")

C:\Users\ana\Documents\Cursos\aluraDataScience\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ana\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

Tempo de processamento: 13.61666202545166 segundos.
- Dimensão do vetor: 1024.


#### 4. Comparando as similaridades das respostas dos modelos

In [11]:
pergunta = "Quero tirar uns dias de folga do trabalho."

emb_pergunta_gemini = gemini_embeddings.embed_query(pergunta)
emb_pergunta_minilm = minilm_embeddings.embed_query(pergunta)
emb_pergunta_bge = bge_embeddings.embed_query(pergunta)

In [13]:
modelos = {
    "Gemini": (emb_pergunta_gemini, embeddings_gemini),
    "MiniLM": (emb_pergunta_minilm, embeddings_minilm),
    "BGE":    (emb_pergunta_bge,    embeddings_bge)
}

In [17]:
print(f"Pergunta: {pergunta}\n")
for nome, (emb_q, emb_docs) in modelos.items():
    similaridade = cosine_similarity([emb_q], emb_docs)[0]
    doc_e_similaridade = sorted(
        zip(textos_teste, similaridade), key=lambda x: x[1], reverse=True
    )
    print(f"--- Ranking para o modelo {nome} ---")
    for i, (doc, sim) in enumerate(doc_e_similaridade[:3], 1):
        print(f" {i}. {doc} - Similaridade: {sim:.2f}")
    print()

Pergunta: Quero tirar uns dias de folga do trabalho.

--- Ranking para o modelo Gemini ---
 1. Qual é a política de férias da nossa empresa? - Similaridade: 0.83
 2. preciso de um relatório de despesas de viagem. - Similaridade: 0.77
 3. Quero entender o processo de avaliação de performance. - Similaridade: 0.75

--- Ranking para o modelo MiniLM ---
 1. Quero entender o processo de avaliação de performance. - Similaridade: 0.50
 2. Onde encontro o código de conduta da organização? - Similaridade: 0.47
 3. Qual é a política de férias da nossa empresa? - Similaridade: 0.47

--- Ranking para o modelo BGE ---
 1. Qual é a política de férias da nossa empresa? - Similaridade: 0.65
 2. preciso de um relatório de despesas de viagem. - Similaridade: 0.65
 3. Onde encontro o código de conduta da organização? - Similaridade: 0.62



##### Gemini e BGE acertaram, com o Gemini tendo mais acurácia.

#### 5. Cached Embeddings

In [20]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

store = LocalFileStore("./cache/")

embedder = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedder,
    document_embedding_cache=store,
    namespace="gemini_cache"
)

C:\Users\ana\Documents\Cursos\aluraDataScience\.venv\Lib\site-packages\langchain\embeddings\cache.py:58: UserWarning: Using default key encoder: SHA‑1 is *not* collision‑resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA‑256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [22]:
textos_para_cache = ["Olá, mundo!", "Testando o cache de embeddings.", "Olá, mundo!"]

start_time = time.time()
embeddings_result_1 = cached_embeddings.embed_documents(textos_para_cache)
end_time = time.time()

print(f"Tempo de processamento: {end_time - start_time:.4f} segundos.")

Tempo de processamento: 0.0132 segundos.


In [25]:
documentos_grandes = [f"Este é o documento de teste número {i}." for i in range(100)]
batch_sizes = [1, 32, 64, 128]

In [26]:
bge_embedder = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

In [27]:
print("Testando eficiência do Batch Processing:")
for batch_size in batch_sizes:
    start_time = time.time()
    num_batches = len(documentos_grandes) // batch_size
    tempo_estimado = num_batches * (0.1 * batch_size) + (len(documentos_grandes) % batch_size) * 0.1
    tempo_real = bge_embedder.client.encode(documentos_grandes, batch_size=batch_size)
    end_time = time.time()

    print(f"Tamanho do lote: {batch_size} -> Tempo: {end_time - start_time:.2f} segundos.")

Testando eficiência do Batch Processing:
Tamanho do lote: 1 -> Tempo: 37.40 segundos.
Tamanho do lote: 32 -> Tempo: 6.58 segundos.
Tamanho do lote: 64 -> Tempo: 4.83 segundos.
Tamanho do lote: 128 -> Tempo: 4.29 segundos.


In [1]:
# Similaridade de cosseno
import numpy as np

# Vetores de exemplo
vetor_a = np.array([1, 2, 3])
vetor_b = np.array([4, 5, 6])

# Cálculo do produto escalar e das normas
produto_escalar = np.dot(vetor_a, vetor_b)
norma_a = np.linalg.norm(vetor_a)
norma_b = np.linalg.norm(vetor_b)

# Cálculo da similaridade de cosseno
similaridade = produto_escalar / (norma_a * norma_b)
print(f'Similaridade de Cosseno: {similaridade}')

Similaridade de Cosseno: 0.9746318461970762
